In [1]:
import numpy as np
import sys 
import time
import pickle
from spam import matrix_to_dict
import pandas as pd
from collections import defaultdict
import pandas as pd

In [4]:
#numpy indexing
x = np.arange(5000).reshape(100,50)
print(x.shape)
print(x[...,1].shape)
print(x[np.arange(0,3), np.arange(0,3)])

(100, 50)
(100,)
[  0  51 102]


In [5]:
def matrix_to_dict(list_fac, list_demands, r):
    """given a set of demand points and a distance matrix 
    converts it to the dictionary for further processing
    
    Inputs: 
        list_demands - list with demand points still not covered
        list_fac - list with facility points still not covered
        r - radius
    
    Returns: 
        dictionary
    """

    dist_matrix = np.genfromtxt('distance_matrix.csv', delimiter=',')
    print(dist_matrix.shape)
    # subsets with list of demands and list of facilities
    dist_matrix = dist_matrix[np.ix_(list_fac, list_demands)]
    print(dist_matrix.shape)
    
    demand = pd.read_csv('demand.csv')

    rows = dist_matrix.shape[0]
    cols = dist_matrix.shape[1]

    lst_array = []
    dict_fac = defaultdict(list)

    for i in range(0,rows): # for each potential facility (n=100)
        for j in range(0,cols): # for each demand point (n=50)
            
            if dist_matrix[i,j]<=r:# and j in list_demands: 
                dict_fac[i].append(j)

    # run through the dictionary and calculate total covered pop
    list_pop = []
    for k,v in dict_fac.items():
        s = 0
        for i in v:
            s = s + demand.iloc[i,2]
        
        list_pop.append([k, s]) # adds count of covered demand points

    stacked_pop = pd.DataFrame(np.vstack(list_pop), columns=['id', 'pop'])

    return dict_fac, stacked_pop 

In [6]:
ss, dd = matrix_to_dict(np.arange(0,50), np.arange(0,10).tolist(), 0.1)
print(len(dd))
ss, dd = matrix_to_dict(np.arange(0,35), np.arange(0,25), 0.1)
print(len(dd))

(100, 50)
(50, 10)
16
(100, 50)
(35, 25)
17


In [7]:
facilities = pd.read_csv('facility.csv')
facilities.columns

Index(['x', 'y', 'facility_id'], dtype='object')

In [8]:
len(facilities.iloc[:,2][~facilities.iloc[:,2].isin([62,21,33])].tolist())

97

In [9]:
def greedy(n_sited):
    """"""

    # inputs 
    #dist_matrix = np.genfromtxt('distance_matrix.csv', delimiter=',') 
    demand = pd.read_csv('demand.csv')
    demand.columns = ['x','y', 'pop', 'demand_id']
    facility = pd.read_csv('facility.csv')
    
    dd, ss = matrix_to_dict(facility.facility_id.tolist(), demand.demand_id.tolist(), r=0.1)
    # sort initial list
    ss.sort_values(by='pop', ascending=False, inplace=True)

    # LOOP 

    # number of sited facilities 
    p = 0 

    # objetive value
    obj = 0 
    
    # decision variables
    covered_demand = []
    sited_candidates = []


    while p < n_sited:
        
        print('shape of ss:', ss.shape)
        
        print('next:', dd[ss.iloc[0,0]])
        print('total items in dict', len(dd))
        
        for i in dd[ss.iloc[0,0]]:
            # add to covered demand from the dictionary 
            covered_demand.append(i)
        print('# of covered demand points:', len(covered_demand))
        print('covered:', covered_demand)
        print('---------------------------')
        
        # calculate total covered demand 
        s = demand.loc[covered_demand,'pop'].sum()
        
        obj = s
            
        p = p + 1
        print('number of facilities:', p)
        print('---------------------------')
        print('objective value:', obj)
        print('---------------------------')

        # add to sited facilities
        sited_candidates.append(ss.iloc[0,0])
        print('sited id:', sited_candidates)
        print('---------------------------')
            
        # recalc and sort the dd again 
        #uncovered_demand = demand.iloc[:,3][~demand.iloc[:,3].isin(covered_demand)].tolist()
        uncovered_demand = demand.loc[~demand.demand_id.isin(covered_demand), 'demand_id'].tolist()
        unsited_facility = facility.loc[~facility.facility_id.isin(sited_candidates), 'facility_id'].tolist()
        #unsited_facility = facilities.iloc[:,2][~facilities.iloc[:,2].isin(sited_candidates)].tolist()
        
        # check 
        if len(list(set(uncovered_demand) & set(covered_demand)))>0:
            print('error')
        
        print(f'number of uncited facilities: {len(unsited_facility)}' )
        print(f'number of uncovered demand: {len(uncovered_demand)}' )
        print('---------------------------')
            
        dd, ss = matrix_to_dict(unsited_facility, uncovered_demand, 0.1)
        
        # remove rows for sited facilities from ss 
        #ss = ss[~ss.id.isin(sited_candidates)]
        #print(ss.shape)
        ss.sort_values(by='pop', ascending=False, inplace=True)
        
        ss.to_csv('matrix_{}'.format(p))
                    
    
    # calculate 
    return obj, ss, dd, covered_demand

In [10]:
obj, ss, dd, covered_demand = greedy(5)

(100, 50)
(100, 50)
shape of ss: (79, 2)
next: [2, 10, 25, 45]
total items in dict 79
# of covered demand points: 4
covered: [2, 10, 25, 45]
---------------------------
number of facilities: 1
---------------------------
objective value: 333
---------------------------
sited id: [62]
---------------------------
number of uncited facilities: 99
number of uncovered demand: 46
---------------------------
(100, 50)
(99, 46)
shape of ss: (76, 2)
next: [6, 20, 21, 26, 37]
total items in dict 76
# of covered demand points: 9
covered: [2, 10, 25, 45, 6, 20, 21, 26, 37]
---------------------------
number of facilities: 2
---------------------------
objective value: 678
---------------------------
sited id: [62, 65]
---------------------------
number of uncited facilities: 98
number of uncovered demand: 41
---------------------------
(100, 50)
(98, 41)
shape of ss: (69, 2)
next: [2, 7, 9, 16, 25]
total items in dict 69
# of covered demand points: 14
covered: [2, 10, 25, 45, 6, 20, 21, 26, 37, 2,

In [11]:
demand = pd.read_csv('demand.csv')
demand.columns = ['x','y', 'pop', 'demand_id']
demand.head()

,x,y,pop,demand_id
0,0.696469,0.120629,77,0
1,0.286139,0.826341,8,1
2,0.226851,0.603060,78,2
3,0.551315,0.545068,6,3
4,0.719469,0.342764,65,4


In [12]:
facility = pd.read_csv('facility.csv')
facility.loc[~facility.facility_id.isin([1,2,3,4,5,6,7]), 'facility_id'].tolist()[:5]

[0, 8, 9, 10, 11]

In [13]:
dist_matrix = np.genfromtxt('distance_matrix.csv', delimiter=',')
print(dist_matrix.shape)
# subsets with list of demands and list of facilities
dist_matrix = dist_matrix[np.ix_(np.arange(0,10), np.arange(0,10))]
print(dist_matrix.shape)

(100, 50)
(10, 10)
